# Neural network test

In [50]:
import os
import sys
import time
 
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'client')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'node')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'shared')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'micropython')))
 
import client
from collections import OrderedDict

In [51]:
import pandas as pd
from pandas import DataFrame
from time import sleep
REFRACTORY_PERIOD = 0.1   # 0.1 seconds

In [52]:
# 每個 ESP8266模組 各代表一個 neurons
# neurons = ['neuron_x1', 'neuron_x2', 'neuron_h1', 'neuron_h2', 'neuron_h3', 'neuron_y'] 
neurons = ['neuron_x1', 'neuron_x2'] 

## Start client

In [53]:
the_client = client.Client()
the_client.start()

while not the_client.status['Is connected']:            
    time.sleep(1)
    print('Node not ready yet.')

My name is Client_366

[Connected: ('192.168.0.105', 9662)]
Sending 313 bytes
Message:
OrderedDict([('command', 'set connection name'), ('correlation_id', '2017-02-02 01:09:53.465100'), ('kwargs', {'name': 'Client_366'}), ('message_id', '2017-02-02 01:09:53.465100'), ('message_type', 'command'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

[Listen to messages]

Data received: 409 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:09:53.465100'), ('message_id', '2017-02-02 01:09:53.630100'), ('message_type', 'result'), ('receiver', 'Client_366'), ('reply_to', 'Hub'), ('result', {'Client_366': "('192.168.0.105', 53710)", 'NodeMCU_1dsc000': "('192.168.0.133', 18611)", 'NodeMCU_f1d30800': "('192.168.0.116', 25879)"}), ('sender', 'Hub'), ('time_stamp', '2017-02-02 01:09:53.630100')])

Node not ready yet.


## Utilities

### List nodes

In [54]:
# Ask Hub for a list of connected nodes
def list_nodes():
    message = {'message_type': 'command',
               'command': 'list connections by name',
               'need_result': True}     

    _, asynch_result = the_client.request('Hub', message) 

    try:
        remote_nodes = sorted(list(asynch_result.get().keys()))

        print ('\n[____________ Connected nodes ____________]\n')        
        print('\nConnected nodes:\n{}\n'.format(remote_nodes)) 
        
        return remote_nodes

    except Exception as e:
        print(e)

In [55]:
def reset_node(node):
    message = {'message_type': 'exec',
               'to_exec': 'import machine;machine.reset()'}
    the_client.request(node, message) 

In [56]:
# reset_node('neuron_x1')

In [57]:
def rename_node(node, new_name):
    message = {'message_type': 'function',
               'function': 'rename',
               'kwargs': {'name': new_name}}
    the_client.request(node, message) 
    
    message = {'message_type': 'function',
               'function': 'set_connection_name'}
    the_client.request(node, message) 
    

def rename_nodes(nodes, neurons):    
    i = 0 
    for node in nodes:
        if node != the_client.node.worker.name:  # exclude client self
            rename_node(node, neurons[i])
            i += 1

In [58]:
def fire(node):
    message = {'message_type': 'function',
               'function': 'fire'}
    the_client.request(node, message) 

def addConnection(node, neuron):
    message = {'message_type': 'function',
               'function': 'addConnection',
               'kwargs': {'neuron_id': neuron}}
    the_client.request(node, message) 
    
def set_connections(node, connections):
    message = {'message_type': 'function',
               'function': 'setConnections',
               'kwargs': {'connections': connections}}
    the_client.request(node, message)     
    
def get_connections(node):
    message = {'message_type': 'function',
               'function': 'getConnections', 
               'need_result': True}
    _, result = the_client.request(node, message) 
    return result.get()    

def setWeight(node, neuron, weight):
    message = {'message_type': 'function',
               'function': 'setWeight',
               'kwargs': {'neuron_id': neuron,
                          'weight': weight,}}
    the_client.request(node, message) 

def setThreshold(node, threshold):
    message = {'message_type': 'function',
               'function': 'setThreshold',
               'kwargs': {'threshold': threshold}}
    the_client.request(node, message) 
        
def getConfig(node):
    message = {'message_type': 'function',
               'function': 'getConfig', 
               'need_result': True}
    _, result = the_client.request(node, message) 
    return result.get()

def getLog(node):
    message = {'message_type': 'function',
               'function': 'getLog', 
               'need_result': True}
    _, result = the_client.request(node, message) 
    return result.get()

def emptyLog(node):
    message = {'message_type': 'function',
               'function': 'emptyLog'}
    the_client.request(node, message)
    
def emptyLogs():
    for neuron in neurons:
        emptyLog(neuron) 
        
# 彙整logs。將所有 neurons 中的 logs merge 在一起，成為一個 Pandas.DataFrame
def mergeLogs():
    logs = []
    
    for neuron in neurons:
        if neuron != the_client.node.worker.name:  # exclude client self
            currentLog = getLog(neuron)
            if currentLog:
                logs += currentLog 
            
    df = DataFrame(list(logs), columns = ['time', 'neuron', 'message']) 
    df.set_index('time', inplace = True)
    df.sort_index(inplace = True)
    
    return df        

In [59]:
# print 出 一個 neuron 中的 Log
def printConfig(neuron):
    print('{0:_^78}\n {1}\n'.format(neuron + " config:", getConfig(neuron)))

## 設定 network config

### Rename nodes

In [60]:
remote_nodes = list_nodes() 
rename_nodes(remote_nodes, neurons)
time.sleep(2)
remote_nodes = list_nodes()

Sending 284 bytes
Message:
OrderedDict([('command', 'list connections by name'), ('correlation_id', '2017-02-02 01:09:55.497100'), ('message_id', '2017-02-02 01:09:55.497100'), ('message_type', 'command'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 409 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:09:55.497100'), ('message_id', '2017-02-02 01:09:55.614100'), ('message_type', 'result'), ('receiver', 'Client_366'), ('reply_to', 'Hub'), ('result', {'Client_366': "('192.168.0.105', 53710)", 'NodeMCU_1dsc000': "('192.168.0.133', 18611)", 'NodeMCU_f1d30800': "('192.168.0.116', 25879)"}), ('sender', 'Hub'), ('time_stamp', '2017-02-02 01:09:55.614100')])


[____________ Connected nodes ____________]


Connected nodes:
['Client_366', 'NodeMCU_1dsc000', 'NodeMCU_f1d30800']

Sending 292 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:09:55.727100'), ('function', 'rename'), ('kwargs', {'name': 'n

### 清空 log files

In [61]:
# 清除所有 neurons 中的 logs  
emptyLogs()

Sending 255 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:09:58.457100'), ('function', 'emptyLog'), ('message_id', '2017-02-02 01:09:58.457100'), ('message_type', 'function'), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Sending 255 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:09:58.568100'), ('function', 'emptyLog'), ('message_id', '2017-02-02 01:09:58.568100'), ('message_type', 'function'), ('receiver', 'neuron_x2'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])



### 設定 connections

In [62]:
addConnection('neuron_x1', 'neuron_x2')
getConfig('neuron_x1')

Sending 298 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:09:58.736100'), ('function', 'addConnection'), ('kwargs', {'neuron_id': 'neuron_x2'}), ('message_id', '2017-02-02 01:09:58.736100'), ('message_type', 'function'), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Sending 277 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:09:58.864100'), ('function', 'getConfig'), ('message_id', '2017-02-02 01:09:58.864100'), ('message_type', 'function'), ('need_result', True), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 394 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:09:58.864100'), ('message_id', '7724036'), ('message_type', 'result'), ('receiver', 'Client_366'), ('reply_to', 'neuron_x1'), ('result', {'inputs': {}, 'output': {'value': 0, 'lasting': 99.99997, 'polarized_time': 7641472}, 'connections': {'neuron_x2': 'neuron_x2'}}), ('sender', 'neuron_x1'), ('tim

{'connections': {'neuron_x2': 'neuron_x2'},
 'inputs': {},
 'output': {'lasting': 99.99997, 'polarized_time': 7641472, 'value': 0}}

### 設定 weights

In [63]:
# hidden layer
setWeight('neuron_x2', 'neuron_x1', 1)  # 設定 neuron_x -> neuron_h1 的 weight = 1

Sending 307 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:09:59.363100'), ('function', 'setWeight'), ('kwargs', {'neuron_id': 'neuron_x1', 'weight': 1}), ('message_id', '2017-02-02 01:09:59.363100'), ('message_type', 'function'), ('receiver', 'neuron_x2'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])



### 設定 thresholds

In [64]:
# input layer 
setThreshold('neuron_x2', 0.9)

Sending 289 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:09:59.597100'), ('function', 'setThreshold'), ('kwargs', {'threshold': 0.9}), ('message_id', '2017-02-02 01:09:59.597100'), ('message_type', 'function'), ('receiver', 'neuron_x2'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])



### 模擬 sensor input，然後查看各 neurons 的 output 狀態
一個 neuron fire 之後，如果沒有持續的輸入可維持 fire 的狀態，則過 5 秒鐘之 neuron 的 output 一定為 0

In [65]:
fire('neuron_x1')

Sending 251 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:09:59.852100'), ('function', 'fire'), ('message_id', '2017-02-02 01:09:59.852100'), ('message_type', 'function'), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])



In [66]:
### 模擬 sensor input，強迫 neuron x 或 y ouput 1
emptyLogs()  # 清除 logs
sleep(REFRACTORY_PERIOD)  # 等電位歸零 
mergeLogs()  # 彙整 logs

Sending 255 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:10:00.076100'), ('function', 'emptyLog'), ('message_id', '2017-02-02 01:10:00.076100'), ('message_type', 'function'), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Sending 255 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:10:00.163100'), ('function', 'emptyLog'), ('message_id', '2017-02-02 01:10:00.163100'), ('message_type', 'function'), ('receiver', 'neuron_x2'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Sending 274 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:10:00.435100'), ('function', 'getLog'), ('message_id', '2017-02-02 01:10:00.435100'), ('message_type', 'function'), ('need_result', True), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 253 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:10:00.435100'), ('message_id', '7725584'), ('message_type', 'result'), (

,neuron,message
time,,
64625,neuron_x2,neuron_x1 is kicking neuron_x2.
64830,neuron_x2,neuron_x2 fires.
64833,neuron_x2,Setting output of neuron_x2 to ACTION_POTENTIAL.


In [67]:
### 模擬 sensor input，強迫 neuron x 或 y ouput 1
emptyLogs()  # 清除 logs
sleep(REFRACTORY_PERIOD)  # 等電位歸零
fire('neuron_x1') # force neuron x1 output 1 and fire.
mergeLogs()  # 彙整 logs

Sending 255 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:10:01.379100'), ('function', 'emptyLog'), ('message_id', '2017-02-02 01:10:01.379100'), ('message_type', 'function'), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Sending 255 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:10:01.477100'), ('function', 'emptyLog'), ('message_id', '2017-02-02 01:10:01.477100'), ('message_type', 'function'), ('receiver', 'neuron_x2'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Sending 251 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:10:01.692100'), ('function', 'fire'), ('message_id', '2017-02-02 01:10:01.692100'), ('message_type', 'function'), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Sending 274 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:10:01.786100'), ('function', 'getLog'), ('message_id', '2017-02-02 01:10:01.786100'), ('message_type', 'f

,neuron,message
time,,
66620,neuron_x2,neuron_x1 is kicking neuron_x2.
66825,neuron_x2,neuron_x2 fires.
66827,neuron_x2,Setting output of neuron_x2 to ACTION_POTENTIAL.
7727061,neuron_x1,neuron_x1 fires.
7727063,neuron_x1,Setting output of neuron_x1 to ACTION_POTENTIAL.


In [68]:
### 模擬 sensor input，強迫 neuron x 或 y ouput 1
emptyLogs()  # 清除 logs
sleep(REFRACTORY_PERIOD)  # 等電位歸零
fire('neuron_x2') # force neuron x2 output 1 and fire.
mergeLogs()  # 彙整 logs

Sending 255 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:10:03.316100'), ('function', 'emptyLog'), ('message_id', '2017-02-02 01:10:03.316100'), ('message_type', 'function'), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Sending 255 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:10:03.416100'), ('function', 'emptyLog'), ('message_id', '2017-02-02 01:10:03.416100'), ('message_type', 'function'), ('receiver', 'neuron_x2'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Sending 251 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:10:03.609100'), ('function', 'fire'), ('message_id', '2017-02-02 01:10:03.609100'), ('message_type', 'function'), ('receiver', 'neuron_x2'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Sending 274 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:10:03.693100'), ('function', 'getLog'), ('message_id', '2017-02-02 01:10:03.693100'), ('message_type', 'f

,neuron,message
time,,
68192,neuron_x2,neuron_x2 fires.
68195,neuron_x2,Setting output of neuron_x2 to ACTION_POTENTIAL.


In [69]:
### 模擬 sensor input，強迫 neuron x 或 y ouput 1
emptyLogs()  # 清除 logs
sleep(REFRACTORY_PERIOD)  # 等電位歸零
fire('neuron_x1') # force neuron x1 output 1 and fire.
fire('neuron_x2') # force neuron x2 output 1 and fire.
mergeLogs()  # 彙整 logs

Sending 255 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:10:04.874100'), ('function', 'emptyLog'), ('message_id', '2017-02-02 01:10:04.874100'), ('message_type', 'function'), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Sending 255 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:10:04.968100'), ('function', 'emptyLog'), ('message_id', '2017-02-02 01:10:04.968100'), ('message_type', 'function'), ('receiver', 'neuron_x2'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Sending 251 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:10:05.187100'), ('function', 'fire'), ('message_id', '2017-02-02 01:10:05.187100'), ('message_type', 'function'), ('receiver', 'neuron_x1'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Sending 251 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:10:05.307100'), ('function', 'fire'), ('message_id', '2017-02-02 01:10:05.307100'), ('message_type', 'fun

,neuron,message
time,,
69852,neuron_x2,neuron_x2 fires.
69855,neuron_x2,Setting output of neuron_x2 to ACTION_POTENTIAL.
69941,neuron_x2,neuron_x1 is kicking neuron_x2.
69946,neuron_x2,neuron_x2 is still in refractory-period.
69950,neuron_x2,neuron_x2 is still in refractory_period at act...
7730568,neuron_x1,neuron_x1 fires.
7730571,neuron_x1,Setting output of neuron_x1 to ACTION_POTENTIAL.


In [70]:
for neuron in reversed(neurons): printConfig(neuron)

Sending 277 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:10:06.924100'), ('function', 'getConfig'), ('message_id', '2017-02-02 01:10:06.924100'), ('message_type', 'function'), ('need_result', True), ('receiver', 'neuron_x2'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 463 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:10:06.924100'), ('message_id', '71353'), ('message_type', 'result'), ('receiver', 'Client_366'), ('reply_to', 'neuron_x2'), ('result', {'inputs': {'neuron_x1': {'value': 1, 'lasting': 500.0, 'kick_time': 69942}}, 'weights': {'neuron_x1': 1}, 'threshold': 0.8999998, 'output': {'value': 1, 'lasting': 99.99997, 'polarized_time': 69856}}), ('sender', 'neuron_x2'), ('time_stamp', '2017-02-02 01:10:07.347100')])
______________________________neuron_x2 config:_______________________________
 {'inputs': {'neuron_x1': {'value': 1, 'lasting': 500.0, 'kick_time': 69942}}, 'weights': {'neuron_x1': 1}, 'threshold': 0.8999998

### Stop the demo

In [71]:
# Stopping
the_client.stop()
the_client = None
print ('\n[________________ Demo stopped ________________]\n')


[________________ Demo stopped ________________]

